In [2]:
%run ../__init__.py

## Data preprocessing with Tfidf

In [3]:
budget_train = pd.read_csv('../data/train_cleaned.csv', index_col=0)
budget_test = pd.read_csv('../data/test_clean.csv', index_col=0)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
budget_train.isnull().sum()

Function                      0
Use                           0
Sharing                       0
Reporting                     0
Student_Type                  0
Position_Type                 0
Object_Type                   0
Pre_K                         0
Operating_Status              0
Object_Description          423
Text_2                       17
SubFund_Description           0
Job_Title_Description         0
Text_3                    70814
Text_4                       18
Sub_Object_Description        0
Location_Description          0
FTE                           0
Function_Description          5
Facility_or_Department        0
Position_Extra              611
Total                         0
Program_Description           2
Fund_Description              0
Text_1                      422
combined_text                 0
dtype: int64

In [5]:
budget_test.isnull().sum()

ID                            0
Object_Description         1752
Program_Description        5253
SubFund_Description       33953
Job_Title_Description     17747
Facility_or_Department    47225
Sub_Object_Description    16452
Location_Description      12748
FTE                           0
Function_Description       3198
Position_Extra            36279
Text_4                    47252
Total                         0
Text_2                    45426
Text_3                    44281
Fund_Description          10478
Text_1                    34717
dtype: int64

In [6]:
for col in budget_test[word_predictors]:
    budget_test[col] = budget_test[word_predictors].loc[:,[col]].fillna(' ')

In [7]:
for col in budget_train[word_predictors]:
    budget_train[col] = budget_train[word_predictors].loc[:,[col]].fillna(' ')

In [8]:
label = LabelEncoder()

In [9]:
class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    Parameters
     ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]
    

In [10]:
class ArrayCaster(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, data):
        return np.transpose(np.matrix(data))

In [12]:
# 400,000+ examples, 14 features
X = budget_train[predictors] 
y = label.fit_transform(budget_train['Operating_Status'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .2, random_state=42)
                                                

In [13]:
# feature union transformer list 
transformer_list = [
    ('Object_Description', Pipeline([
                ('selector', ItemSelector(key='Object_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_2', Pipeline([
                ('selector', ItemSelector(key='Text_2')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('SubFund_Description', Pipeline([
                ('selector', ItemSelector(key='SubFund_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Job_Title_Description', Pipeline([
                ('selector', ItemSelector(key='Job_Title_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_3', Pipeline([
                ('selector', ItemSelector(key='Text_3')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_4', Pipeline([
                ('selector', ItemSelector(key='Text_4')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Sub_Object_Description', Pipeline([
                ('selector', ItemSelector(key='Sub_Object_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Location_Description', Pipeline([
                ('selector', ItemSelector(key='Location_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Function_Description', Pipeline([
                ('selector', ItemSelector(key='Function_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Facility_or_Department', Pipeline([
                ('selector', ItemSelector(key='Facility_or_Department')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Position_Extra', Pipeline([
                ('selector', ItemSelector(key='Position_Extra')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Program_Description', Pipeline([
                ('selector', ItemSelector(key='Program_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Fund_Description', Pipeline([
                ('selector', ItemSelector(key='Fund_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_1', Pipeline([
                ('selector', ItemSelector(key='Text_1')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
#     ('FTE', Pipeline([
#                 ('selector', ItemSelector(key='FTE')),
#                 ('scaler', StandardScaler()),
#                 ('caster', ArrayCaster()),
#             ])),
#     ('Total', Pipeline([
#                 ('selector', ItemSelector(key='Total')),
#                 ('scaler', StandardScaler()),
#                 ('caster', ArrayCaster()),
#             ])),
]

### Don't include numerical columns in final feature union since it decreased scores and increased log loss- in general, in the original data, numerical features had a lot of missing values and some incorrect values for data that were there

In [16]:
# create pipeline
model_pipeline = Pipeline([
    ("union", FeatureUnion(transformer_list=transformer_list)),
    ("clf", LogisticRegression()),
])
params = {'clf__C': np.logspace(-2,4,5)}

In [17]:
gs = GridSearchCV(model_pipeline, params, n_jobs=1)

gs.fit(X_train, y_train)

In [19]:
gs.best_estimator_

Pipeline(steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('Object_Description', Pipeline(steps=[('selector', ItemSelector(key='Object_Description')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='...ty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

In [20]:
gs.best_params_

{'clf__C': 10000.0}

In [14]:
X_test = budget_test

In [15]:
op_stat_model = model_pipeline.fit(X_train, y_train)

In [16]:
op_stat_model.score(X_train, y_train)

0.98528203959140714

In [17]:
val_proba = op_stat_model.predict_proba(X_val)

In [18]:
predict_proba = op_stat_model.predict_proba(X_test)

In [19]:
label.classes_

array(['Operating_Status__Non-Operating',
       'Operating_Status__Operating, Not PreK-12',
       'Operating_Status__PreK-12 Operating'], dtype=object)

In [20]:
log_loss(y_val, val_proba)

0.047490405186274731

In [21]:
from sklearn.externals import joblib

In [22]:
joblib.dump(op_stat_model, 'op_stat_model.pkl')

['op_stat_model.pkl']

In [23]:
def score_eval(model, X_val, y_val):
    preds = model.predict(X_val)
    proba = model.predict_proba(X_val)
    return {
        'log_loss': log_loss(y_val, proba),
        'accuracy': accuracy_score(y_val, preds),
        'confmat': confusion_matrix(y_val, preds),
        'clf_rep': classification_report(y_val, preds)
    }

In [24]:
results = score_eval(op_stat_model, X_val, y_val)

print('-'*60)
for key, value in results.items():
    print(key)
    print(value)
    print('-'*60)

------------------------------------------------------------
log_loss
0.0474904051863
------------------------------------------------------------
accuracy
0.984460877386
------------------------------------------------------------
confmat
[[ 9158    25   495]
 [   35  1321   373]
 [  307     9 68333]]
------------------------------------------------------------
clf_rep
             precision    recall  f1-score   support

          0       0.96      0.95      0.96      9678
          1       0.97      0.76      0.86      1729
          2       0.99      1.00      0.99     68649

avg / total       0.98      0.98      0.98     80056

------------------------------------------------------------


In [25]:
classes = label.classes_

In [26]:
op_stat_probability = pd.DataFrame(predict_proba, columns=classes)

In [27]:
op_stat_probability.head()

,Operating_Status__Non-Operating,"Operating_Status__Operating, Not PreK-12",Operating_Status__PreK-12 Operating
0,0.010871,0.020641,0.968488
1,0.010554,0.000055,0.989391
2,0.009930,0.003089,0.986981
3,0.009926,0.003074,0.987000
4,0.002271,0.000066,0.997663


In [28]:
op_stat_probability.to_csv('op_stat_probability.csv')